In [1]:
import pandas as pd
from pyomo.environ import *

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
# dem = pd.read_csv('C:/Jupyter/TESIS/instancias/demanda_n15_k200_t60.txt')
# preco = pd.read_csv('C:/Jupyter/TESIS/instancias/preco_n15_k200_t60.txt')

# dem = pd.read_excel('C:/tesis_wilmer/tesis/instancias/demanda_n7_k150_t60.xlsx')
# preco = pd.read_excel('C:/tesis_wilmer/tesis/instancias/preco_n7_k150_t60.xlsx')

#dem = pd.read_csv('C:/tesis_wilmer/tesis/instancias/demanda_n15_k200_t60.txt')
#preco = pd.read_csv('C:/tesis_wilmer/tesis/instancias/preco_n15_k200_t60.txt')

dem_ini = pd.read_csv('C:/tesis_wilmer/tesis/dem_ini.txt')
dem_anex = pd.read_csv('C:/tesis_wilmer/tesis/demanda_anex_t.txt')

# preco_ini = pd.read_csv('C:/tesis_wilmer/tesis/preco_ini.txt')
# preco_anex = pd.read_csv('C:/tesis_wilmer/tesis/preco_anex_n2.txt')

dem = pd.concat([dem_ini, dem_anex], axis=0)
preco = pd.read_csv('C:/tesis_wilmer/tesis/preco_ini.txt')
# preco = pd.concat([preco_ini, preco_anex], axis=0)



In [3]:
dem_cor = dem.set_index(['origem', 'destino', 'classe','periodo']).T.to_dict('records')[0]
preco_cor = preco.set_index(['origem', 'destino', 'classe']).T.to_dict('records')[0]

In [4]:
cap = 700 #capacidad del tren
n = 7 #numero de estaciones
kk = 150 #numero de classes
tt = 120 #numero de periodos

In [ ]:
# instanciar o modelo
model = AbstractModel()

# Escalares
model.n = Param(initialize = n)
model.i = Param(initialize = n)
model.j = Param(initialize = n)
model.k = Param(initialize = kk)
model.t = Param(initialize = tt)
model.Q = Param(initialize = cap)

# conjuntos
model.I = RangeSet(1, model.i)
model.J = RangeSet(1, model.j)
model.K = RangeSet(1, model.k)
model.T = RangeSet(1, model.t)

#parâmetros indexados
model.P = Param(model.I, model.J, model.K, initialize = preco_cor)
model.d = Param(model.I, model.J, model.K, model.T, initialize = dem_cor)

# variáveis de decisão
model.X = Var(RangeSet(0, model.i), model.J, model.K, model.T, within = NonNegativeIntegers)
model.Y = Var(RangeSet(0, model.i), model.J, model.K, model.T, within = NonNegativeIntegers)
model.A = Var(RangeSet(0, model.i), within = NonNegativeIntegers)

# função objetivo
def fo1(model):
    return sum(model.P[i,j,k]*(model.X[i,j,k,t]) for i in model.I for j in model.J for k in model.K for t in model.T if i<j)
model.OBJ1 = Objective(rule = fo1, sense = maximize)

# restrições
def cap1(model, i):
    return model.A[i] ==  model.A[i-1] - sum((model.X[i-1,j,k,t]) for j in model.J for k in model.K for t in model.T if j>=i) + sum((model.X[j,i,k,t]) for j in model.J for k in model.K for t in model.T if j<i)
model.ConstrainCap1 = Constraint(model.I, rule = cap1)

def cap2(model, i):
    if i < model.n:
        return sum((model.X[i,j,k,t]) for j in model.J for k in model.K for t in model.T if i<j) <= model.A[i]
    else:
        return Constraint.Skip
model.ConstrainCap2 = Constraint(model.I, rule = cap2)

def authorization(model, i, j, k, t):
    if i < j:
        return model.Y[i,j,k,t] >= model.X[i,j,k,t]
    else:
        return Constraint.Skip
model.ConstrainAuthorization = Constraint(model.I, model.J, model.K, model.T, rule = authorization)

def classe(model, i, j, k, t):
    if i < j and k < model.k:
        return model.Y[i,j,k,t] <= model.Y[i,j,k+1,t]
    else:
        return Constraint.Skip
model.ConstrainClasse = Constraint(model.I, model.J, model.K, model.T, rule = classe)

def assignment(model, i, j, k, t):
    if i < j:
        return model.X[i,j,k,t] <= model.d[i,j,k,t]
    else:
        return Constraint.Skip
model.ConstrainAssignment = Constraint(model.I, model.J, model.K, model.T,rule = assignment)

def assignment0(model, j, k, t):
    return model.X[0,j,k, t] == 0
model.ConstrainAssignment0 = Constraint(model.J, model.K, model.T, rule = assignment0)

def authorization0(model, j, k, t):
    return model.Y[0,j,k,t] == 0
model.ConstrainAuthorization0 = Constraint(model.J, model.K, model.T, rule = authorization0)

def cap0(model):
    return model.A[0] == model.Q
model.ConstrainCap0 = Constraint(rule = cap0)

instance = model.create_instance()
# instance.pprint()

opt = SolverFactory('cplex')
results = opt.solve(instance, tee=True)
print(results)

In [ ]:
print('Valor da função objetivo: ', '\033[1m' + '\033[32m' + str(value(instance.OBJ1)) + '\033[0m')

print('')

# print('Valores das variáveis de decisão: ')
lista = []
for i in range(1,n+1):
    for j in range(1,n+1):
        for k in range(1,kk+1):
            for t in range(1,tt+1):
                if i<j and value(instance.Y[i,j,k,t])!=0:
                    # print(i,j,k, ' - ',value(instance.Y[i,j,k]), ' - ', value(instance.X[i,j,k]) )
                    lista.append([str(i) + '-' + str(j), k, t, value(instance.P[i,j,k]), value(instance.d[i,j,k,t]) , value(instance.X[i,j,k,t]), value(instance.Y[i,j,k,t]) ])

a = pd.DataFrame(lista, columns=['o-d','classe','Periodo','preco','demanda','Assignments','Authorizations'])
print(a)

In [7]:
# print('')
# print('Valores das capacidades em cada nó: ')

# for i in range(1, n+1):
#     print(i , ' - ', value(instance.A[i]))

# print('')
# tabela_pivote = pd.pivot_table(a[a['classe']==1],  values='Authorizations', index=['origem','classe'], columns=['destino'], aggfunc='sum', margins=True, margins_name='Total')
# tabela_pivote2 = pd.pivot_table(a[a['classe']==1],  values='Assignments', index=['origem','classe'], columns=['destino'], aggfunc='sum', margins=True, margins_name='Total')

# tabela_pivote3 = pd.pivot_table(a[a['classe']==2],  values='Authorizations', index=['origem','classe'], columns=['destino'], aggfunc='sum', margins=True, margins_name='Total')
# tabela_pivote4 = pd.pivot_table(a[a['classe']==2],  values='Assignments', index=['origem','classe'], columns=['destino'], aggfunc='sum', margins=True, margins_name='Total')

# result_1 = tabela_pivote + tabela_pivote2
# result_2 = tabela_pivote3 + tabela_pivote4

# print(result_1, '\n \n', result_2)

# a = pd.DataFrame(lista, columns=['o-d','classe','Periodo','preco','demanda','Assignments','Authorizations'])
# a.to_excel('modelo_1_1.xlsx', index=False)

# corregit instancias
# dem_cor_1 = {(i,j,k,t): dem_cor[(i,j,k,t)] if (i,j,k,t) in dem_cor else 0  for i in range(1, n+1) for j in range(1, n+1) for k in range(1, kk+1) for t in range(1, tt+1)}
# preco_cor_1 = {(i,j,k): preco_cor[(i,j,k)] if (i,j,k) in preco_cor else 0  for i in range(1, n+1) for j in range(1, n+1) for k in range(1, kk+1)}